## Install Required Packages

In [103]:
%%bash
python3.11 -m pip install --upgrade pip
pip install numpy
pip install scipy
pip install keras
pip install tensorflow
pip install pandas

## Imports

In [104]:
import numpy as np
from scipy import io
import os
import pickle
import pandas as pd

# Import function to get covariate matrix that includes spike history from previous bins
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history

# Import metrics
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho

# Import decoder functions
from Neural_Decoding.decoders import LSTMDecoder


## User Inputs (Modify as Necessary)

In [105]:
binned_dir = "Data_Binned" # Name of directory that stores binned results
results_dir = "Results" # Name of directory that will store results

bins_before = 5 # How many bins of neural data prior to the output are used for decoding (fewer bins is faster)
bins_current = 1 # Whether to use concurrent time bin of neural data
bins_after = 5 # How many bins of neural data after the output are used for decoding (fewer bins is faster)

training_range = [0, 0.85] # Range of training data
testing_range = [0.85, 1] # Range of testing data

## LSTM

In [106]:
# Get all .pickle files
parent_dir = os.path.dirname(os.path.abspath("__file__"))
data_dir = os.path.join(parent_dir, binned_dir)
files = next(os.walk(data_dir))[2]

results_dir = os.path.join(parent_dir, results_dir)
if not os.path.exists(results_dir):
    os.mkdir(results_dir)

for f in files:
    if not '.pickle' in f:
        continue

    dataset = f.split('.')[0]
    print('dataset: ' + dataset)

    # Skip dataset if results already exist
    if os.path.exists(results_dir + '/' + dataset + '.csv'):
        continue

    with open(data_dir + "/" + f, 'rb') as res:
        neural_data, pos_binned, markers = pickle.load(res, encoding='latin1')

    X = get_spikes_with_history(neural_data, bins_before, bins_after, bins_current)
    y = pos_binned

    num_examples = X.shape[0]
    training_set = np.arange(int(np.round(training_range[0] * num_examples)) + bins_before, int(np.round(training_range[1] * num_examples)) - bins_after)
    testing_set = np.arange(int(np.round(testing_range[0] * num_examples)) + bins_before, int(np.round(testing_range[1] * num_examples)) - bins_after)

    # Get training data
    X_train = X[training_set, :, :]
    y_train = y[training_set, :]

    # Get testing data
    X_test = X[testing_set, :, :]
    y_test = y[testing_set, :]

    # Z-score "X" inputs. 
    X_train_mean = np.nanmean(X_train, axis=0)
    X_train_std = np.nanstd(X_train, axis=0)
    X_train = np.divide((X_train - X_train_mean), X_train_std, where=X_train_std[1] != 0)
    X_test = np.divide((X_test - X_train_mean), X_train_std, where=X_train_std[1] != 0)


    # Zero-center outputs
    y_train_mean = np.nanmean(y_train, axis=0)
    y_train = y_train - y_train_mean
    y_test = y_test - y_train_mean

    # Declare model
    model_lstm = LSTMDecoder(units=400, dropout=0, num_epochs=20, verbose=1)

    # Fit model
    model_lstm.fit(X_train, y_train)

    # Get predictions
    y_predicted_lstm = model_lstm.predict(X_test)

    # Get results
    R2_lstm = get_rho(y_test, y_predicted_lstm) ** 2
    print('R2:', R2_lstm)
    markers = [m[0][0].replace('30Hz_', '') for m in markers]
    print(markers)

    results = {label: r2 for label, r2 in zip(markers, R2_lstm)}
    df = pd.DataFrame(results, index = [0])
    df.to_csv(results_dir + '/' + dataset + '.csv')

dataset: Br_S1U_Superficial
dataset: Br_PFU_Superficial
dataset: YeCon_M1U_Anterior
dataset: Br_CSF_Superficial
dataset: RyCon_S1F_Deep
dataset: YeNb_M1F_Posterior
dataset: YeCon_S1U_Anterior
dataset: RyCon_M1F_Posterior
dataset: YeCon_S1U_Deep
dataset: RyCon_S1U_Intermediate
dataset: RyCon_S1U_Deep
dataset: Br_CSF_Intermediate
dataset: Br_S1U_Deep
dataset: Br_S1U_Posterior
dataset: YeCon_S1U_Posterior
dataset: RyNb_M1U_Posterior
Epoch 1/20
401/401 [==============================] - 15s 34ms/step - loss: 14.2211 - accuracy: 0.5291
Epoch 2/20
401/401 [==============================] - 15s 38ms/step - loss: 4.2215 - accuracy: 0.6302
Epoch 3/20
401/401 [==============================] - 15s 37ms/step - loss: 2.1020 - accuracy: 0.6879
Epoch 4/20
401/401 [==============================] - 15s 38ms/step - loss: 1.2083 - accuracy: 0.7294
Epoch 5/20
401/401 [==============================] - 15s 37ms/step - loss: 1.0549 - accuracy: 0.7533
Epoch 6/20
401/401 [==============================] - 1